In [1]:
import copy
import numpy as np

import torch
import torchvision
from torch import nn
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.utils import save_image
from torchvision.datasets import MNIST
import os

In [2]:
%cd datasets
!bash get_datasets.sh
%cd ..

/home/acauligi/cs_231n/cs231n-project/datasets
/home/acauligi/cs_231n/cs231n-project


In [3]:
from datasets.data_utils import get_CIFAR10_data
data = get_CIFAR10_data(num_training=100, num_validation=10, num_test=100)
data['X_train'][:1].shape

(1, 3, 32, 32)

In [4]:
from src.configs import BallEncoder, BallDecoder

In [5]:
dim_in = data['X_train'][0].shape
dim_z = 6
dim_u = 0

channels_enc = [3, 8, 8] # vector of length N+1 specifying # of channels for each conv layer
ff_shape = [32, 32]   # vector specifying shape of feedforward network

conv_activation = torch.nn.ReLU()
ff_activation = torch.nn.ReLU()

n_channels = len(channels_enc) - 1
kernel_enc = [2,3]
stride= [2] * n_channels
padding= [2] * n_channels
pool = [2, 2] * n_channels

enc = BallEncoder(dim_in, dim_z, channels_enc, ff_shape, \
                 conv_activation, ff_activation, \
                 kernel_enc, stride, padding, pool)

In [6]:
inp = torch.from_numpy(data['X_train'][1:4]).float()
mu, var = enc(inp)
mu

tensor([[0.3701, 0.8016, 0.0000, 4.0176, 0.0000, 0.0000],
        [1.5026, 2.0516, 0.0000, 5.2417, 0.0000, 0.0000],
        [1.4796, 2.1794, 0.0000, 3.0253, 0.0000, 0.0000]],
       grad_fn=<SplitBackward>)

In [7]:
channels_dec = [8, 8, dim_in[0]]
kernel_dec = [2, 2]
dec =  BallDecoder(dim_z, dim_in, channels_dec, ff_shape, kernel_dec, stride, padding, ff_activation, conv_activation)

dec(mu).shape

torch.Size([3, 3, 32, 32])